In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
import os
os.chdir('/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔1(切割)/')
os.listdir() #確認目錄內容

['segment_19.m4a',
 'segment_8.m4a',
 'segment_16.m4a',
 'segment_10.m4a',
 'segment_12.m4a',
 'segment_18.m4a',
 'segment_13.m4a',
 'segment_20.m4a',
 'segment_4.m4a',
 'segment_6.m4a',
 'segment_17.m4a',
 'segment_0.m4a',
 'segment_9.m4a',
 'segment_14.m4a',
 'segment_11.m4a',
 'segment_1.m4a',
 'segment_5.m4a',
 'segment_7.m4a',
 'segment_2.m4a',
 'segment_3.m4a',
 'segment_15.m4a',
 'segment_0.wav',
 'model_computational_graph',
 'model_computational_graph.png']

In [45]:
!pip install transformers
!pip install datasets

# 試圖微調 HuBERT，但發現微調也是要labels的

### 範例延伸的嘗試

In [46]:
import torch
import torchaudio
from transformers import AutoProcessor, HubertForCTC

PATH = '/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔1(切割)/segment_0.m4a'
test_PATH = '/content/drive/My Drive/Colab_Notebooks/speech_research/日文語音原始/A/音檔5.m4a'

waveform, sample_rate = torchaudio.load(PATH)

# 把 rate從 44100轉成 16000
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)

# 如果 waveform 是雙聲道，需要轉單聲道
if waveform.size(0) > 1:
    waveform = waveform.mean(dim=0)  # 取平均值，[2, 19424] => [19424]


# 載入對應的 processor 和 model
model_name = "facebook/hubert-large-ls960-ft"
processor = AutoProcessor.from_pretrained(model_name)
model = HubertForCTC.from_pretrained(model_name)

# 凍結 CNN 層參數
for name, param in model.named_parameters():
  if "Conv" in name:  # 假設 CNN 層參數名稱包含 'cnn'
    param.requires_grad = False
  if "conv" in name:
    param.requires_grad = False

# 把 wave轉成 hubert接受的格式
inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")

with torch.no_grad():
    # logits是 model對每一格的每個類別的預測機率
    # 這時 output並沒有 loss，因為沒有 labels
    outputs = model(**inputs)
    logits = outputs.logits
# 把每一格最有可能的 labels輸出
predicted_ids = torch.argmax(logits, dim=-1)

outputs

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-

CausalLMOutput(loss=None, logits=tensor([[[  7.6259, -22.7104, -22.6982,  ...,  -4.8661,  -6.4280,  -7.1026],
         [  7.0148, -22.1836, -22.2451,  ...,  -4.7322,  -6.2848,  -6.9824],
         [  7.0982, -22.4304, -22.5271,  ...,  -5.0531,  -6.7796,  -7.1665],
         ...,
         [  4.5161, -23.9012, -23.8980,  ...,  -8.2140,  -8.6098,  -4.1410],
         [  3.6143, -22.3419, -22.5539,  ...,  -6.8472,  -7.8395,  -4.1056],
         [  4.5293, -22.2382, -22.5334,  ...,  -5.2996,  -6.2803,  -4.8710]]]), hidden_states=None, attentions=None)

In [47]:
predicted_ids

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18, 11,  0,  7,  0,  6,  0,  4,
          0,  7,  0,  0,  4,  4]])

In [48]:
transcription = processor.batch_decode(predicted_ids)
print("labels = WATASHI")
print("predicted_ids_trans: ", transcription[0])

labels = WATASHI
predicted_ids_trans:  WHAT A


In [49]:
from IPython.display import Audio

Audio(PATH)

In [50]:
# 該 audio對應的真 label
inputs["labels"] = processor(text='WATASHI', return_tensors="pt").input_ids

# compute loss
# 用 labels和 預測的計算
loss = model(**inputs).loss
round(loss.item(), 2)

23.15

In [51]:
inputs.labels

tensor([[18,  7,  6,  7, 12, 11, 10]])

In [52]:
transcription = processor.batch_decode(inputs.labels)
transcription[0]

'WATASHI'

# 針對學長姐資料集做預測

# 套用前專題的轉成單聲道功能
# 套用前專題的轉日文字功能，再轉羅馬拼音做成ids

## 製作 dataset, labels set(待修正，目前靜態)

In [53]:
label_list = ['WATASHI', 'WATASHI TACHI', 'ANATA', 'ANO HITO', 'ANO KATA', 'MINASAN', 'SENSEI', 'KYOSHI', 'GAKU SEI', 'KAISHA IN', 'SHA IN', 'GIN KOIN', 'ISHA', 'KEN KYU SHA', 'ENJINIA', 'DAI GA KU', 'BYO IN', 'DENKI', 'DARA', 'DONATA', '~ SAI', 'NA N SAI',  'O IKUTSU']
all_label = 'WATASHI,WATASHI TACHI,ANATA,ANO HITO,ANO KATA,MINASAN,SENSEI,KYOSHI,GAKU SEI,KAISHA IN,SHA IN,GIN KOIN,ISHA,KEN KYU SHA,ENJINIA,DAI GA KU,BYO IN,DENKI,DARA,DONATA,~ SAI,NA N SAI,O IKUTSU'

In [54]:
import pandas as pd
import torchaudio

df = pd.DataFrame(columns=['audio_path', 'text_labels'])

for index in range(0, 23):
  PATH=f'/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔3(切割)/segment_{index}.m4a'
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)

for index in range(0, 23):
  PATH=f'/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔5(切割)/segment_{index}.m4a'
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)

<ipython-input-54-09ad3f5ce0ba>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-54-09ad3f5ce0ba>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-54-09ad3f5ce0ba>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-54-09ad3f5ce0ba>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels':

In [55]:
[df['text_labels'][index] for index in range(len(label_list))]

['WATASHI',
 'WATASHI TACHI',
 'ANATA',
 'ANO HITO',
 'ANO KATA',
 'MINASAN',
 'SENSEI',
 'KYOSHI',
 'GAKU SEI',
 'KAISHA IN',
 'SHA IN',
 'GIN KOIN',
 'ISHA',
 'KEN KYU SHA',
 'ENJINIA',
 'DAI GA KU',
 'BYO IN',
 'DENKI',
 'DARA',
 'DONATA',
 '~ SAI',
 'NA N SAI',
 'O IKUTSU']

# 轉換成 dataset，為將來的 batch

In [56]:
def process_waveforms(batch):
  waveform, sample_rate = torchaudio.load(batch["audio_path"])

  # 把 rate從 44100轉成 16000
  if sample_rate != 16000:
      resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
      waveform = resampler(waveform)

  # 如果 waveform 是雙聲道，需要轉單聲道。給 4GE用
  if waveform.size(0) > 1:
      waveform = waveform.mean(dim=0)  # 取平均值，[2, 19424] => [19424]

  batch["speech_array"] = waveform
  batch["sample_rate"] = 16000

  batch["labels"] = batch['text_labels']

  return batch

'''
  input_value = processor(waveform, sampling_rate=16000, return_tensors="pt")
  # 差 id label就可以放 model input
  TEXT = 轉日文字(PATH)
    label_text = 轉羅馬拼音(TEXT)
    input_value['label'] = processor(text=label_text, return_tensors="pt").input_ids
'''


'\n  input_value = processor(waveform, sampling_rate=16000, return_tensors="pt")\n  # 差 id label就可以放 model input\n  TEXT = 轉日文字(PATH)\n    label_text = 轉羅馬拼音(TEXT)\n    input_value[\'label\'] = processor(text=label_text, return_tensors="pt").input_ids\n'

In [57]:
from datasets import Dataset

df = Dataset.from_pandas(df)
df = df.map(process_waveforms, remove_columns=df.column_names)

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

In [58]:
df

Dataset({
    features: ['speech_array', 'sample_rate', 'labels'],
    num_rows: 46
})

# 了解 processor

# 需要了解 train時 epoch, batch的運作

In [59]:
import torch

class Trainer:
  def __init__(self, model, processor, optimizer):
    self.model = model
    self.processor = processor
    self.optimizer = optimizer
    self.total_loss = 0
    self.model.eval() # 不打這行，model也本來就預設evaluate mode

  def training_one_batch(self, batch):
    self.model.train()
    inputs = self.processor(batch["speech_array"],
                  sampling_rate=16000,
                  return_tensors="pt",
                  padding=True,
                  truncation=True,
                  max_length=160000)
    inputs["labels"] = self.processor(text=batch['labels'], return_tensors="pt", padding=True).input_ids

    # forward and predict & calculate CTC_loss
    outputs = self.model(**inputs)

    self.total_loss = 0
    loss = outputs.loss
    self.total_loss += loss.item()

    # Backward and optimize
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    return batch

  def evaluate(self, batch):
    self.model.eval()
    inputs = self.processor(batch["speech_array"],
                sampling_rate=16000,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=160000)
    inputs["labels"] = self.processor(text=batch['labels'], return_tensors="pt", padding=True).input_ids

    with torch.no_grad():
      outputs = self.model(**inputs)

    return outputs

# 創造 Trainer的實例

In [60]:
from torch import optim

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)

trainer = Trainer(model, processor, optimizer)

In [61]:
# Training loop
num_epochs = 5
batch_size = 2
num_batches = 56 // batch_size


for epoch in range(num_epochs):

  output = df.map(trainer.training_one_batch, batched=True, batch_size=batch_size)
  print(trainer.total_loss)

  # Print loss or other metrics after each epoch
  average_loss = trainer.total_loss / num_batches
  print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss}")

Parameter 'function'=<bound method Trainer.training_one_batch of <__main__.Trainer object at 0x786b9ddc8fa0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

46.809532165527344
Epoch 1/5, Average Loss: 1.6717690059116908


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

48.44920349121094
Epoch 2/5, Average Loss: 1.7303286961146764


...

35.73017501831055
Epoch 3/5, Average Loss: 1.2760776792253767


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

30.879146575927734
Epoch 4/5, Average Loss: 1.1028266634259904


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

33.70399475097656
Epoch 5/5, Average Loss: 1.203714098249163


In [62]:
segment = 20
sample_batch = df[segment]
output = trainer.evaluate(sample_batch)
output

CausalLMOutput(loss=tensor(33.6324), logits=tensor([[[  4.5828, -20.2541, -19.9700, -19.9204,   0.0957,  -1.8406,  -1.2149,
           -1.6630,  -3.1490,  -1.9459,  -0.2904,  -0.8691,   1.6253,  -3.6722,
           -1.6651,  -4.7671,  -1.7061,  -3.9643,  -3.8792,  -1.7668,  -3.8369,
           -2.3157,  -3.0137,  -4.0151,  -3.9673,  -4.4431,  -1.0111,  -2.6649,
           -3.3329,  -3.7798,  -5.4334,  -2.4595],
         [  4.1911, -18.8867, -18.4936, -18.6746,  -0.6839,  -1.7838,   0.7176,
           -1.7105,  -2.7518,  -2.9346,  -0.1021,  -0.6567,   4.7535,  -4.3213,
           -1.6776,  -5.2148,  -2.2220,  -6.1732,  -4.7842,  -1.0580,  -3.5166,
           -2.8860,  -3.0963,  -3.7860,  -5.1780,  -5.2892,  -0.1331,  -2.1711,
           -2.2659,  -3.3725,  -4.8903,  -0.5821],
         [  2.4988, -16.9332, -16.4121, -16.6817,  -2.1218,  -3.4197,   3.5508,
           -2.0307,  -4.7150,  -1.4430,  -1.9427,  -0.0961,   3.9825,  -3.2514,
            1.1295,  -3.6549,  -3.2625,  -4.6531,  -3.

In [63]:
logits = output.logits
predicted_ids = torch.argmax(logits, dim=-1)

In [64]:
print('predicted_ids: ', predicted_ids)
print('inputs.labels: ', df[segment]['labels'])
transcription = processor.batch_decode(predicted_ids)
transcription[0]

predicted_ids:  tensor([[ 0, 12, 12,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0]])
inputs.labels:  ~ SAI


'SA'

# 找其他人的聲音試試看
# 音檔的切割前後留白要多一點，聲音收錄的不完整，導致預測怪怪的

In [65]:
folder = 4

all_label
# test_PATH

# make dataframe(table)
df = pd.DataFrame(columns=['audio_path', 'text_labels'])
for index in range(0, 23):
  PATH=f'/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔{folder}(切割)/segment_{index}.m4a'
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)

# transfer to dataset and process to {speech_array, attention, text_labels}
df = Dataset.from_pandas(df)
df = df.map(process_waveforms, remove_columns=df.column_names)

<ipython-input-65-406ade93dd8e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-65-406ade93dd8e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-65-406ade93dd8e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labels': label_list[index]}, ignore_index=True)
<ipython-input-65-406ade93dd8e>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'audio_path': PATH, 'text_labe

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [72]:
# use the Trainer.evalute()
segment = 10
sample_batch = df[segment]
test_output = trainer.evaluate(sample_batch)
test_output.loss

tensor(46.4496)

In [73]:
test_predicted_ids = torch.argmax(test_output.logits, dim=-1)
test_predicted_ids

tensor([[ 0,  0, 21, 21,  0,  0,  0,  0, 10,  0,  0,  9,  9,  0,  0,  0,  0,  0,
         26, 26,  0,  0,  0,  0,  0,  0,  0,  8,  8,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0, 10,  0,  0,  9,  0,  0,  0,  4,  0,  0]])

In [74]:
test_transcription = processor.batch_decode(test_predicted_ids)
print('predicted labels: ', test_transcription[0])
print('labels: ', label_list[segment])

predicted labels:  GINKOIN
labels:  SHA IN


In [75]:
PATH=f'/content/drive/My Drive/Colab_Notebooks/speech_research/切割音檔/音檔{folder}(切割)/segment_{segment}.m4a'
Audio(PATH)

In [70]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

hubert.masked_spec_embed True
hubert.feature_extractor.conv_layers.0.conv.weight False
hubert.feature_extractor.conv_layers.0.conv.bias False
hubert.feature_extractor.conv_layers.0.layer_norm.weight False
hubert.feature_extractor.conv_layers.0.layer_norm.bias False
hubert.feature_extractor.conv_layers.1.conv.weight False
hubert.feature_extractor.conv_layers.1.conv.bias False
hubert.feature_extractor.conv_layers.1.layer_norm.weight False
hubert.feature_extractor.conv_layers.1.layer_norm.bias False
hubert.feature_extractor.conv_layers.2.conv.weight False
hubert.feature_extractor.conv_layers.2.conv.bias False
hubert.feature_extractor.conv_layers.2.layer_norm.weight False
hubert.feature_extractor.conv_layers.2.layer_norm.bias False
hubert.feature_extractor.conv_layers.3.conv.weight False
hubert.feature_extractor.conv_layers.3.conv.bias False
hubert.feature_extractor.conv_layers.3.layer_norm.weight False
hubert.feature_extractor.conv_layers.3.layer_norm.bias False
hubert.feature_extractor.c

In [71]:
from transformers import AutoProcessor, HubertForCTC
import torch

model = HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft")
processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model.config.ctc_zero_infinity = True

# 凍結 CNN 層參數
for name, param in model.named_parameters():
  if "Conv" in name:  # 假設 CNN 層參數名稱包含 'cnn'
    param.requires_grad = False
  if "conv" in name:
    param.requires_grad = False

model_load_path = '/content/drive/My Drive/Colab_Notebooks/speech_research/Huert_base_on_English/hubert_finetuned.pth'
model.load_state_dict(torch.load(model_load_path))

Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForCTC: ['hubert.encoder.pos_conv_embed.conv.weight_g', 'hubert.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForCTC were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-

<All keys matched successfully>